In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
tr = pd.read_csv("input/cmc_train.csv")
ts = pd.read_csv("input/cmc_test.csv")
df = pd.concat([tr, ts], axis=0)
df['contraceptive'] = df['contraceptive'].replace(1, 'no-use')
df['contraceptive'] = df['contraceptive'].replace(2, 'long-term')
df['contraceptive'] = df['contraceptive'].replace(3, 'short-term')

In [18]:
d = {}
total = 0
for wife_age in sorted(tr['wifes_age'].unique()):
    var_1 = tr[tr['wifes_age'] == wife_age]
    d['wifes_age_' + str(wife_age)] = {}
    for wife_edu in sorted(tr['wifes_edu'].unique()):
        var_2 = var_1[var_1['wifes_edu'] == wife_edu]
        d['wifes_age_' + str(wife_age)]['wife_edu_' + str(wife_edu)] = var_2.shape[0]
        total += var_2.shape[0]
#         print('wifes_age_' + str(wife_age), 'wife_edu' + str(wife_edu), var_2.shape[0])

for k1 in d:
    for k2 in d[k1]:
        d[k1][k2] = d[k1][k2]/total

pd.DataFrame(d)

,wifes_age_1,wifes_age_2,wifes_age_3,wifes_age_4
wife_edu_1,0.014567,0.016023,0.018937,0.051712
wife_edu_2,0.063365,0.060452,0.064093,0.040058
wife_edu_3,0.084487,0.080117,0.059723,0.055353
wife_edu_4,0.075018,0.090313,0.121631,0.104151


In [49]:
def probability(data, variable_name):
    p = {}
    proba_key = 'proba'
    p[variable_name] = []
    p[proba_key] = []
    total = 0
    
    for variable_value in data[variable_name].unique():
        event_name = '{}_{}'.format(variable_name, variable_value)
        frequency = data[data[variable_name] == variable_value].shape[0]
        p[variable_name].append(event_name)
        p[proba_key].append(frequency)
        total += frequency

    for i, v in enumerate(p[proba_key]):
        p[proba_key][i] = v / total    

    return pd.DataFrame(p)

probability(tr, 'wifes_edu')

,proba,wifes_edu
0,0.227968,wifes_edu_2
1,0.101238,wifes_edu_1
2,0.279680,wifes_edu_3
3,0.391114,wifes_edu_4


In [90]:
from itertools import product

def join(data, variables):
    p = {}
    proba_key = 'proba'
    p[proba_key] = []
    total = 0
  
    variables_events = []
    for variable_name in variables:
        variable_events = []
        for variable_value in data[variable_name].unique():
            variable_events.append((variable_name, variable_value))
        variables_events.append(variable_events)

    for variables_values in product(*variables_events):
        expr = ' & '.join(map(lambda x: '({} == {})'.format(x[0], x[1]), variables_values))
        print(data.query(expr))
#         print('&'.join(list(variables_values)))
#         for x in variables_values:
#             print(x)
        print()
#     print(list())
#     print(' & '.join(list(map(list, ))))
#     print(' && '.join(map(list, product(*variables_events))))
    
#     for variable_name in variables:
#         p[variable_name] = []
#         for variable_value in data[variable_name].unique():
#             event_name = '{}_{}'.format(variable_name, variable_value)
#             frequency = data[data[variable_name] == variable_value].shape[0]
#             p[variable_name].append(event_name)
#             p[proba_key].append(frequency)
#             total += frequency

#     for i, v in enumerate(p[proba_key]):
#         p[proba_key][i] = v / total    
#     print(p)
#     return pd.DataFrame(p)

join(tr, ['wifes_edu', 'wifes_age'])

      wifes_age  wifes_edu  husbands_edu  n_children  wifes_rel  \
0             1          2             3           3          1   
33            1          2             3           1          1   
54            1          2             2           2          1   
57            1          2             3           0          1   
59            1          2             3           1          1   
61            1          2             3           1          1   
63            1          2             4           1          1   
78            1          2             4           1          1   
85            1          2             3           0          1   
93            1          2             2           3          1   
95            1          2             3           0          1   
98            1          2             4           2          1   
117           1          2             3           1          0   
148           1          2             2           2          

In [39]:
def conditional(data, hypothesis, evidence):
    p = {}
    p[evidence] = {}
    evidence_name = '_'.join(evidence.split('_')[:-1])
    evidence_value = int(evidence.split('_')[-1])
    evidence_data = data[data[evidence_name] == evidence_value]
    total = 0

    # find frequencies
    for hypothesis_value in evidence_data[hypothesis].unique():
        hypothesis_name = '{}_{}'.format(hypothesis, hypothesis_value)
        hypothesis_data = evidence_data[evidence_data[hypothesis] == hypothesis_value]
        p[evidence][hypothesis_name] = hypothesis_data[hypothesis].shape[0]
        total += hypothesis_data[hypothesis].shape[0]

    # find probabilities
    for k1 in p:
        for k2 in p[k1]:
            p[k1][k2] = p[k1][k2] / total

    return pd.DataFrame(p)

conditional(tr, 'wifes_working', 'wifes_age_2')

,wifes_age_2
wifes_working_0,0.289086
wifes_working_1,0.710914


In [ ]:
def marginal(variable):
    pass